In [155]:
import numpy as np
import pandas as pd
import pulp

<img src="./images/example_lp_formulation.jpg" width=500 height=500 align="left"/>

## Define Problem

In [156]:
prob = pulp.LpProblem("ProductMix",pulp.LpMaximize)

In [157]:
prob

ProductMix:
MAXIMIZE
None
VARIABLES

## Create decision variables

In [158]:
products = ['Desks', 'Chairs', 'Bookcases']

In [159]:
product_dvs = {}
for product in products:
    var = pulp.LpVariable(name=product,lowBound=0,)
    product_dvs[product] = var

In [160]:
product_dvs

{'Desks': Desks, 'Chairs': Chairs, 'Bookcases': Bookcases}

## Cutting hours capacity constraint

In [161]:
cutting_hours = {
    'Desks' : 0.8,
    'Chairs' : 0.4,
    'Bookcases' : 0.4
}
available_cutting_hours = 100

In [162]:
expression = pulp.LpAffineExpression()
for product in products:
    expression += cutting_hours[product] * product_dvs[product]

In [163]:
expression = expression <= available_cutting_hours
cutting_hours_constraint = pulp.LpConstraint(expression,name="CuttingHoursCapacity")

In [164]:
cutting_hours_constraint

0.4*Bookcases + 0.4*Chairs + 0.8*Desks + -100.0 = 0

In [165]:
prob += cutting_hours_constraint

## Man hours capacity constraint

In [166]:
man_hours = {
    'Desks' : 5,
    'Chairs' : 3,
    'Bookcases' : 3
}
available_man_hours = 650

In [167]:
expression = pulp.LpAffineExpression()
for product in products:
    expression += man_hours[product] * product_dvs[product]

In [168]:
expression = expression <= available_man_hours
man_hours_constraint = pulp.LpConstraint(expression,name="ManHoursCapacity")

In [169]:
man_hours_constraint

3*Bookcases + 3*Chairs + 5*Desks + -650 = 0

In [170]:
prob += man_hours_constraint

## Warehouse capacity constraint

In [171]:
floor_space_requirements = {
    'Desks' : 1,
    'Chairs' : 0.7,
    'Bookcases' : 1.1
}
available_floor_space = 140

In [172]:
expression = pulp.LpAffineExpression()
for product in products:
    expression += floor_space_requirements[product] * product_dvs[product]

In [173]:
expression = expression <= available_floor_space
floor_space_constraint = pulp.LpConstraint(expression,name="WarehouseCapacity")

In [174]:
floor_space_constraint

1.1*Bookcases + 0.7*Chairs + 1*Desks + -140.0 = 0

In [175]:
prob += floor_space_constraint

In [176]:
prob

ProductMix:
MAXIMIZE
None
SUBJECT TO
CuttingHoursCapacity: 0.4 Bookcases + 0.4 Chairs + 0.8 Desks = 100

ManHoursCapacity: 3 Bookcases + 3 Chairs + 5 Desks = 650

WarehouseCapacity: 1.1 Bookcases + 0.7 Chairs + Desks = 140

VARIABLES
Bookcases Continuous
Chairs Continuous
Desks Continuous

## Objective Function

## Maximize Profit
## $30 X_{Desks} + 16 X_{Chairs} + 25 X_{Bookcases}$

In [177]:
product_profit = {
    'Desks' : 30,
    'Chairs' : 16,
    'Bookcases' : 5
}

In [178]:
obj_expression = pulp.LpAffineExpression()
for product in products:
    obj_expression += product_profit[product] * product_dvs[product]

In [179]:
obj_expression

5*Bookcases + 16*Chairs + 30*Desks + 0

In [180]:
prob += obj_expression

In [181]:
prob

ProductMix:
MAXIMIZE
5*Bookcases + 16*Chairs + 30*Desks + 0
SUBJECT TO
CuttingHoursCapacity: 0.4 Bookcases + 0.4 Chairs + 0.8 Desks = 100

ManHoursCapacity: 3 Bookcases + 3 Chairs + 5 Desks = 650

WarehouseCapacity: 1.1 Bookcases + 0.7 Chairs + Desks = 140

VARIABLES
Bookcases Continuous
Chairs Continuous
Desks Continuous

In [189]:
solver = pulp.get_solver('GLPK_CMD')

In [190]:
prob.solve(solver=solver)

1

In [ ]:
for product in products:
    value = product_dvs[product].value()
    print(f"{product} = {value}")

In [ ]:
for c_name in prob.constraints:
    print(c_name,prob.constraints[c_name].value())

# Solvers

In [185]:
solver_list = pulp.listSolvers()
solver_list

['GLPK_CMD',
 'PYGLPK',
 'CPLEX_CMD',
 'CPLEX_PY',
 'CPLEX_DLL',
 'GUROBI',
 'GUROBI_CMD',
 'MOSEK',
 'XPRESS',
 'PULP_CBC_CMD',
 'COIN_CMD',
 'COINMP_DLL',
 'CHOCO_CMD',
 'PULP_CHOCO_CMD',
 'MIPCL_CMD',
 'SCIP_CMD']

In [186]:
solver_list = pulp.listSolvers(onlyAvailable=True)
solver_list

['GLPK_CMD', 'SCIP_CMD']

In [191]:
solver = pulp.get_solver('SCIP_CMD')

In [192]:
solver

In [197]:
solver.to_dict()

{'solver': 'SCIP_CMD',
 'mip': True,
 'msg': True,
 'keepFiles': False,
 'maxNodes': None}

# How does it work ?

<img src="./images/model_solver_eco_system.jpg" width=600 height=500 align="left"/>